# data collection
This notebook is responsible for collecting CrossFit 2018 Open Leaderboard data and athelete profile data *as represented at the time of data collection*.

## imports
The below is just a set of import statements required to run the code in this notebook. A description for the purpose of each import statement should be commented above it.

In [1]:
#sql connector
import pymysql as pms
#time recording and sleeping
from time import time, sleep
#browser automation
from selenium import webdriver
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC

## database credentials
In order to connect to a local MySQL database, the block below runs to read the username, password, database name, and host required to establish a connection.

In [2]:
db_user = ""
db_pass = ""
db_name = ""
db_host = ""
with open("database_credentials.txt") as f:
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_name = f.readline().strip()
    db_host = f.readline().strip()

## test database connection
This short snippet is going to attempt to connect to the database and drop out without doing anything. This is just to make sure the credentials and PyMySQL are working properly.

In [3]:
def get_connect():
    """
    Returns a database connection object using the default params
    specified in the database_credentials file.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)

In [4]:
try:
    con = get_connect()
    print("Connected.")
    with con.cursor() as cur:
        print("Got database cursor. Can make queries within here.")
finally:
    if con:
        con.close()
        print("Connection closed.")

Connected.
Got database cursor. Can make queries within here.
Connection closed.


## urls
These urls/variables can be used to jump to pages where leaderboard data is available.

In [5]:
default_url = "https://games.crossfit.com/leaderboard/open/2018?division=1&region=0&scaled=0&sort=0&occupation=0&page=1"
custom_url = "https://games.crossfit.com/leaderboard/open/2018?division={}&region={}&scaled={}&sort={}&occupation={}&page={}"

#this map would be used to substitute values into the
#custom url string at position "division={}" in place
#of the "{}"
map_division = {
    "men": 1,
    "women": 2,
    "team": 11,
    #men aged (35-39) inclusive
    "m35-39": 18,
    #women aged (35-39) inclusive
    "w35-39": 19,
    "m40-44": 12,
    "w40-44": 13,
    "m45-49": 3,
    "w45-49": 4,
    "m50-54": 5,
    "w50-54": 6,
    "m55-59": 7,
    "w55-59": 8,
    "m60+": 9,
    "w60+": 10,
    #boys aged (16-17) inclusive
    "b16-17": 16,
    #girls aged (16-17) inclusive
    "g16-17": 17,
    "b14-15": 14,
    "g14-15": 15
}

## getting custom url values
Although the URL custom attributes can be harded, the more robust solution is to write a browser automation step, prior to the main data collection, that acquires the custom url attributes corresponding to each filter. This is done below.

The goal is to obtain all of the maps like the one above in a more robust manner.

Also, in the below step, it's important to note that the year and competition can also be used to filter results. However, at the current time, regional data is not available for 2018 (the open just finished). Furthermore, previous open leaderboard have different HTML structure (would require additional scraping code), and I really only care about 2018. Additionally, Rx'd/scaled, per-workout, occupation, and region are also available filtering criteria.

**Rx'd/scaled and occupation**
I don't care about these for the time being. This repo will only consider non-specific occupation and Rx'd athletes.

**per-workout and region**
I'll be able to do this filtering on my own (hypothetically). In order to do so, the region and per-workout scores will be scraped from the leaderboard and filtered on later in other notebooks.

In [6]:
filters = ["division", "region"]
print("HTML filters which will be used for filtering:\n{}".format(filters))

HTML filters which will be used for filtering:
['division', 'region']


Below we're going to go to the leaderboard (autonomously) and scrape the IDs [CrossFit](https://games.crossfit.com/leaderboard/open/2018?division=1&region=0&scaled=0&sort=0&occupation=0&page=1) uses for divisions and regions. Although I don't need to use the same IDs, it can only help to use the same mappings. When we're talking about IDs here, I mean the numeric values that would be used to plug into the `"{}"` occurences in the `custom_url` string above.

Here's the documentation I use for [Selenium](http://selenium-python.readthedocs.io/locating-elements.html).

In [7]:
#attempt database connect
try:
    con = get_connect()
    with con.cursor() as cur:
        #create division and region tables if they don't exist
        sql = """
        CREATE TABLE IF NOT EXISTS region (
            id INT PRIMARY KEY,
            region VARCHAR(24) NOT NULL
        );
        CREATE TABLE IF NOT EXISTS division (
            id INT PRIMARY KEY,
            division VARCHAR(16) NOT NULL
        );
        """
        cur.execute(sql)
        
        #attempt to get regions and divisions
        result_counts = [-1, -1]
        sql = """
        SELECT * FROM {};
        """
        for i in range(len(filters)):
            cur.execute(sql.format(filters[i]))
            result = cur.fetchall()
            #store number of results
            result_counts[i] = len(result)
            #output results
            print("==== {} results({}) ====".format(filters[i], result_counts[i]))
            print(", ".join([col[0] for col in cur.description]))
            for j in range(result_counts[i]):
                print("{}: {}".format(j, result[j]))
        
        #if results for both are not empty, the values have
        #already been scraped, so skip this
        if result_counts[0] < 1 or result_counts[1] < 1:
            #Store id, region/div pairs as tuples (id_0, region_0/div_0)
            #the below entries will have 2 lists of such tuples, 1 for
            #each filter_id
            entries = []
            #spin up browser
            driver = webdriver.Chrome()
            driver.get(default_url)
            
            #iterate over useful filters
            for i in range(len(filters)):
                #ids are formatted with control- as a prefix
                dropdown = driver.find_element_by_id("control-" + filters[i])
                options = dropdown.find_elements_by_tag_name("option")
                #aggregate entries
                #also, this is making 2 calls to o.get_attribute("value"), and this
                #could be done "more efficiently" without the list comprehension
                entries.append([(int(o.get_attribute("value")), o.get_attribute("innerText"))
                            for o in options if o.get_attribute("value") != ""])
                #print(entries[i])
            
            #close driver
            driver.close()
        
            #write entries to file
            sql = """
            INSERT INTO {}(id, {}) VALUES
                {}
            """
            for i in range(len(filters)):
                cur.execute(sql.format(filters[i], filters[i],
                                       ",\n".join(["({}, '{}')".format(e[0], e[1]) for e in entries[i]])))
            #commit inserts
            con.commit()
finally:
    if con:
        con.close()

==== division results(19) ====
id, division
0: (1, 'Men')
1: (2, 'Women')
2: (3, 'Men (45-49)')
3: (4, 'Women (45-49)')
4: (5, 'Men (50-54)')
5: (6, 'Women (50-54)')
6: (7, 'Men (55-59)')
7: (8, 'Women (55-59)')
8: (9, 'Men (60+)')
9: (10, 'Women (60+)')
10: (11, 'Team')
11: (12, 'Men (40-44)')
12: (13, 'Women (40-44)')
13: (14, 'Boys (14-15)')
14: (15, 'Girls (14-15)')
15: (16, 'Boys (16-17)')
16: (17, 'Girls (16-17)')
17: (18, 'Men (35-39)')
18: (19, 'Women (35-39)')
==== region results(19) ====
id, region
0: (0, 'Worldwide')
1: (5, 'Canada West')
2: (6, 'Central East')
3: (9, 'Mid Atlantic')
4: (10, 'North Central')
5: (11, 'North East')
6: (14, 'South Central')
7: (15, 'South East')
8: (17, 'South West')
9: (18, 'Canada East')
10: (19, 'West Coast')
11: (20, 'Asia')
12: (21, 'Australasia')
13: (22, 'Europe North')
14: (23, 'Europe Central')
15: (24, 'Europe South')
16: (25, 'Africa Middle East')
17: (26, 'Central America')
18: (27, 'South America')


C:\Users\unknown\AppData\Local\conda\conda\envs\crossfit_open_2018\lib\site-packages\pymysql\cursors.py:103: Warning: (1050, "Table 'division' already exists")
  return self._nextset(False)


## checkout filter values
Now that we've ensured the filters are in the database, let's grab them and create mappings. These will be necessary for the athlete table.

In [8]:
#this map will contain maps for each filter
filter_maps = {}
try:
    con = get_connect()
    with con.cursor() as cur:
        sql = """
        SELECT * FROM {};
        """
        for f in filters:
            #add filter map
            filter_maps[f] = {}
            #get results
            cur.execute(sql.format(f))
            result = cur.fetchall()
            #store results
            for r in result:
                #create mapping from div/region -> id
                filter_maps[f][r[1]] = r[0]
finally:
    if con:
        con.close()

## hard-coded values
At the time of data collection, the 2018 CrossFit Open has ended. With this, we're making the assumption the total number of pages for each leaderboard will not change (this is not necessarily 100%, but I'm assuming it's very close to 100%). The leaderboard is structured in a way such that if a page is requested beyond the total number of leaderboard pages available for a specific filter, it redirects back to the first leaderboard page.

Although there are different ways to handle this, in order to know when to stop scraping for a specific filter, we'll just scrape the index of the last available page, which is available at the bottom of each leaderboard. Additionally, the current index for each filter will also scraped, defaulting to the first leaderboard page: 1.

In [9]:
try:
    con = get_connect()
    with con.cursor() as cur:
        #create database to store these worldwide-division indices
        #(could easily be stored in a flat file instead)
        sql = """
        CREATE TABLE IF NOT EXISTS worldwide_division_pages (
            division_id INT PRIMARY KEY,
            FOREIGN KEY (division_id)
                REFERENCES division(id)
                ON DELETE CASCADE,
            curr_page INT NOT NULL DEFAULT 1,
            last_page INT NOT NULL DEFAULT -1
        )
        """
        cur.execute(sql)
        
        #check if table has been populated already in a previous run
        sql = """
        SELECT * FROM worldwide_division_pages;
        """
        cur.execute(sql)
        result = cur.fetchall()
        if len(result) == 0:
            #cross-populate division id's from division table
            sql = """
            INSERT INTO worldwide_division_pages (division_id)
                SELECT id FROM division;
            """
            cur.execute(sql)
            con.commit()
            
            #grab option IDs from database
            #this could be done by just collecting them from the browser, the values
            #are supposed to be identical
            sql = """
            SELECT division_id FROM worldwide_division_pages;
            """
            cur.execute(sql)
            result = cur.fetchall()
            ids = [r[0] for r in result]
            print(ids)
                     
            #scrape last_page values for each division
            driver = webdriver.Chrome()
            driver.get(default_url)
            
            #grab division selectable and store in browser
            inject_store_select = """
            window.division_select = document.getElementById("control-division");
            """
            driver.execute_script(inject_store_select)
            
            last_pages = []
            #iterate over ids
            for i in ids:
                #force select element to change to new dropdown option
                inject_change_select = """
                window.division_select.value = {};
                window.division_select.dispatchEvent(new Event("change"));
                """.format(i)
                driver.execute_script(inject_change_select)
                #wait for page to update
                sleep(2)
                #grab last page text from the bottom
                last_pages.append(int(
                    driver.find_element_by_class_name("nums")
                        .find_elements_by_tag_name("a")[-1]
                        .get_attribute("innerText")
                ))
            
            #write updates to file in bulk
            sql = """
            INSERT INTO worldwide_division_pages(division_id, last_page) VALUES
                {}
                ON DUPLICATE KEY UPDATE last_page = VALUES(last_page);
            """
            cur.execute(
                sql.format(
                    ",\n".join(["({}, '{}')".format(ids[i], last_pages[i]) for i in range(len(ids))])
                )
            )
            con.commit()
            
            #close driver
            driver.close()
finally:
    if con:
        con.close()

C:\Users\unknown\AppData\Local\conda\conda\envs\crossfit_open_2018\lib\site-packages\pymysql\cursors.py:166: Warning: (1050, "Table 'worldwide_division_pages' already exists")
  result = self._query(query)


### additional fixed filters
Below are the hard-coded values for scaled, occupation, sort, and workout type.

In [10]:
region = 0
scaled = 0
sort = 0
occupation = 0

## leaderboard/athlete scraping
Scraping the Open leaderboard data is only half of the data required per athlete. The remaining data will be collected from their athlete profile, containing statistics for their Back Squat, Fran, and other CrossFit staples. This will be done **after** the Open data for a specific division is completely finished. Therefore, the scraping process pipeline from this point forward is as follows:
* for each division:
    * scrape all leaderboard data
    * for each athlete:
        * scrape profile data

### reading in the pages to be scraped from the Open leaderboard
Below the pages left to be scraped will be collected from the database and sorted in order from least athletes to most athletes **per division**. Divisions which have complete Open leaderboard data already scraped will have a current page value exceeding the last page value by 1 (`curr_page = 3`, `last_page = 2`).

In [17]:
try:
    con = get_connect()
    with con.cursor() as cur:
        sql = """
        SELECT * FROM worldwide_division_pages;
        """
        cur.execute(sql)
        #sort results based on last_page value
        division_pages = sorted(cur.fetchall(), key=lambda tup: tup[2])
finally:
    if con:
        con.close()

In [18]:
division_pages

[(15, 1, 32),
 (14, 1, 40),
 (10, 1, 45),
 (17, 1, 47),
 (9, 1, 55),
 (16, 1, 63),
 (11, 1, 74),
 (8, 1, 76),
 (7, 1, 91),
 (6, 1, 124),
 (5, 1, 167),
 (4, 1, 239),
 (3, 1, 337),
 (13, 1, 398),
 (12, 1, 568),
 (19, 1, 619),
 (18, 1, 894),
 (2, 1, 3441),
 (1, 1, 4552)]